# Optimal charging station location on a cycleway


Consider a long linear cycle path  as Vento (VENezia-TOrino), or the Danube cycle path. The cycle path usually runs along the banks of a river with scarse tourist interest. However, from the main course of the cycle path it is possible to reach places of tourist interest by making small detours.  

The rapid growth of e-bike ridership is proposing the problem of deploying a suitable charging infrastructure. The charging stations should be placed in strategic positions so as to guarantee a coverage of the whole cycle path. However, since the charging operations require a non negligible time, the charging station should be positioned in places where alternative activities could be carried out, as restaurants, museums, swimming pool, or other amenities. These places, called **Points of Interest (POI)**  are not on the main trajectory of the cyclepath, but the bikers must deviate to reach them.

We assume that the bikers enter the cyclepath where it begins and finish the trip ath the end of the cyclepath. When they enter the battery is fully charged. Moreover, the bikers will make all possible deviations in order to visit all the POIs.







## Formulation

To support the formulation we make use of a graph with $2n+2$ nodes.
Nodes $s$ and $t$ represent the extremes of the cyclepath.

Nodes

*   $L = \{1,\ldots,n\}$:  locations along the cyclepath from which bikers deviate


*   $H = \{1',\ldots, n'\}$:  POIs that may host a charging station.

*   $\{s,t\}$: extremes of the cyclepath.

Arcs

*  $\{(i,i+1), (i+1, i) 1,\ldots,n \}$: arcs of the cyclepath in the two directions

*  $\{(i,i'), (i',i), i\in L, i'\in H\}$: arcs representing the deviations from the cyclepath

### Example of the graph
![picture](https://drive.google.com/file/d/1A4dSP-4K1E3jwaab4A8JT21TKD7FINaK/view?usp=drive_link)

### Parameters

*   $d_{i,i+1}, d_{i+1,i}, i=1,\ldots,n-1$: energy consumption between consecutive nodes,
*   $d_{i,i'}, d_{i',i}, i \in L, i'\in H$: energy consumption for the deviations
*   $c_i'$: cost of installing a charging station in site $i' \in H$
*   $\Delta$: maximum energy consumption between consecutive charging stations

We assume that $d_{s,1} d_{1,s} = d_{n,t} = d_{t,n} = 0$.
Note that the energy consumption on each arc can be different in the two directions, depending on the orography.








## Problem 1

Consider a biker that traverses the cyclepath from $s$ to $t$, determine in which nodes of $H$ to install the charging stations so that the maximum energy consumption between two consecutive charging stations is no more than $\Delta$ and minimize the overall cost.



## Problem 2

Consider a biker that traverses the cyclepath from $t$ to $s$, determine in which nodes of $H$ to install the charging stations so that the maximum energy consumption between two consecutive charging stations is no more than $\Delta$ and minimize the overall cost.

### Comments
Do you obtain the same solution? Why?

### Recommendation
The problems can be solved by a simple formulation. The solution of complex problems as the TSP is not necessary.
Do not use external libraries apart from those presented in class.



In [ ]:
#import libraries

!pip install mip
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
import numpy as np
import math
import networkx as nx
import matplotlib.pyplot as plt

## Solution Problem 1

In [2]:
#data PROBLEM 1

n = 15  # number of nodes on the main course
n1 = 15 #number of touristic sites
delta = 50  # max distance before recharge
s = 0   # starting point
t = n  # destination
consumption = [20, 32, 11, 37, 7, 14, 22, 5, 35, 17, 23, 3, 26, 24] # consumption (in Wh) between two consecutive location along the main course
consumption_deviation = [1.1, 0.7, 0.4, 0.9, 2.1, 1.8, 0.5, 0.4, 1.6, 2.5, 1.4, 0.8, 2.0, 1.3, 0.1] # consumption (in Wh) of the deviation
inst_cost = [1492, 1789, 1914, 1861, 1348, 1769, 1123, 1432, 1564, 1818, 1901, 1265, 1642, 1712, 1756] #cost (in €) of installation of a charging point related to the node

In [ ]:
# Graph visualization

g1 = nx.DiGraph()

s = 's'
t = 't'

nodes = [i for i in range(1, n + n1 + 1)]  # set of nodes without s and t

x_coordinates = [20, 40, 72, 83, 120, 127, 141, 163, 168, 203, 220, 243, 246, 272, 296]  # vector where each element is the cumulative sum of distances up to the i-th node

coord1 = {}  # associate coordinates to each node
for i in nodes:
    if i <= n:
        coord1[i] = (x_coordinates[i - 1], 0)  # main nodes in a horizontal line
    else:
        coord1[i] = (x_coordinates[i - n - 1], consumption_deviation[i - 16] + 1)  # deviation nodes above main nodes

arcs = [(i, j) for i in nodes for j in nodes if (j == n + i or (j == i + 1 and j <= 15))]  # set of arcs, without s and t

for i in range(1, n):  # add weights to arcs for labels
    g1.add_edge(i, i + 1, weight=consumption[i - 1])
for i in range(1, n1 + 1):
    g1.add_edge(i, n + i, weight=consumption_deviation[i - 1])
    g1.add_edge(n + i, i, weight=consumption_deviation[i - 1])

nodes += [s, t]  # add nodes s and t, and respective arcs with null weight
arcs += [(s, 1), (15, t)]
coord1[s] = (0, 0)
coord1[t] = (x_coordinates[n - 1] + 20, 0)
g1.add_edge(s, 1, weight=0)
g1.add_edge(n, t, weight=0)

g1.add_nodes_from(nodes)

node_colors1 = ['yellow' for node in g1.nodes()]

# Dictionary for node labels
labels = {}
for i in range(1, n + 1):
    labels[i] = str(i)
for i in range(n + 1, n + n1 + 1):
    labels[i] = f"{i - n}'"
labels[s] = 's'
labels[t] = 't'

plt.figure(figsize=(25, 6))
nx.draw(g1, pos=coord1, with_labels=True, labels=labels, node_color=node_colors1, node_size=500, font_size=10)
edge_labels = nx.get_edge_attributes(g1, 'weight')
nx.draw_networkx_edge_labels(g1, pos=coord1, edge_labels=edge_labels, font_color='blue')
plt.show()


In [ ]:
def distance(i, j):
    N = j - i
    if i >= 16:
        current_POI = i - 16
        dist = 0
        for a in range(N):
            dist += POI_distances[current_POI]
            current_POI += 1
        return dist
    else:
        return 0

POI_distances = [21.8, 33.1, 12.3, 40, 10.9, 16.3, 22.9, 7, 39.1, 20.9, 25.2, 5.8, 29.3, 25.4]

import mip

# model
m = mip.Model()

# variables
x = [m.add_var(var_type=mip.BINARY) for i in range(n1)]

# constraints
for i in range(16, 31):
    for j in range(i + 1, 31):
        dist = distance(i, j)
        m.add_constr(mip.xsum(x[k] for k in range(i - 15, j - 16)) >= (1 / 1000) * (dist - 50))

# objective function
m.objective = mip.minimize(mip.xsum(inst_cost[i] * x[i] for i in range(n1)))

# model optimization
m.optimize()

# print the result
n_installed = 0
for i in range(len(x)):
    n_installed += x[i].x

print("Number of charging stations installed:", n_installed)
print("Total cost:", m.objective_value)
for i in range(n1):
    if x[i].x >= 0.99:
        print(f"Charging station installed at POI {i+1}'")


In [ ]:
# Solution visualization on the graph: POIs with charging stations in green

highlighted_nodes = [17, 19, 20, 22, 23, 25, 27, 29]

node_colors1 = ['green' if node in highlighted_nodes else 'yellow' for node in g1.nodes()]

labels = {}
for i in range(1, n + 1):
    labels[i] = str(i)
for i in range(n + 1, n + n1 + 1):
    labels[i] = f"{i - n}'"
labels[s] = 's'
labels[t] = 't'

plt.figure(figsize=(25, 6))
nx.draw(g1, pos=coord1, with_labels=True, labels=labels, node_color=node_colors1, node_size=500, font_size=10)
edge_labels = nx.get_edge_attributes(g1, 'weight')
nx.draw_networkx_edge_labels(g1, pos=coord1, edge_labels=edge_labels, font_color='blue')
plt.show()


## Solution Problem 2

In [6]:
#data PROBLEM 2

n = 15  # number of nodes on the main course
n1 = 15 #number of touristic sites
delta = 50  # max distance before recharge
s = 0   # starting point
t = n  # destination
consumption = [12, 24, 7, 8, 35, 21, 19, 9, 31, 15, 21, 6, 23, 29] # consumption (in Wh) between two consecutive location along the main course
consumption_deviation = [1.7, 2.1, 1.1, 0.3, 0.4, 2.3, 0.2, 0.9, 2.1, 4.3, 5.2, 0.1, 2.8, 0.9, 0.1] # consumption (in Wh) of the deviation

inst_cost = [1492, 1789, 1914, 1861, 1348, 1769, 1123, 1432, 1564, 1818, 1901, 1265, 1642, 1712, 1756]

In [ ]:
# Graph visualization with updated weights

consumption.reverse()
consumption_deviation.reverse()

g2 = nx.DiGraph()

s = 's'
t = 't'

nodes = [i for i in range(1, n + n1 + 1)]

x_coordinates = [20, 49, 72, 78, 99, 114, 145, 154, 173, 194, 229, 237, 244, 268, 280]

coord2 = {}
for i in nodes:
    if i <= n:
        coord2[i] = (x_coordinates[i - 1], 0)
    else:
        coord2[i] = (x_coordinates[i - n - 1], consumption_deviation[i - 16] + 1)

arcs = [(i, j) for i in nodes for j in nodes if (j == n + i or (j == i + 1 and j <= 15))]
for i in range(1, n):
    g2.add_edge(i + 1, i, weight=consumption[i - 1])
for i in range(1, n1 + 1):
    g2.add_edge(i, n + i, weight=consumption_deviation[i - 1])
    g2.add_edge(n + i, i, weight=consumption_deviation[i - 1])

nodes += [s, t]
arcs += [(s, 1), (15, t)]
coord2[s] = (0, 0)
coord2[t] = (x_coordinates[n - 1] + 20, 0)
g2.add_edge(1, s, weight=0)
g2.add_edge(t, n, weight=0)

g2.add_nodes_from(nodes)

node_colors2 = ['yellow' for node in g2.nodes()]

labels = {}
for i in range(1, n + 1):
    labels[i] = str(i)
for i in range(n + 1, n + n1 + 1):
    labels[i] = f"{i - n}'"
labels[s] = 's'
labels[t] = 't'

plt.figure(figsize=(25, 6))
nx.draw(g2, pos=coord2, with_labels=True, labels=labels, node_color=node_colors2, node_size=500, font_size=10)
edge_labels = nx.get_edge_attributes(g2, 'weight')
nx.draw_networkx_edge_labels(g2, pos=coord2, edge_labels=edge_labels, font_color='blue')
plt.show()

consumption.reverse()
consumption_deviation.reverse()


In [ ]:
def distance(i, j):
    N = j - i
    if i >= 16:
        current_POI = i - 16
        dist = 0
        for a in range(N):
            dist += POI_distances[current_POI]
            current_POI += 1
        return dist
    else:
        return 0

POI_distances = [30, 26.7, 8.9, 26.3, 24.5, 37.4, 12, 20.1, 23.5, 37.7, 8.7, 8.4, 27.2, 15.8]

import mip

# model
m = mip.Model()

# variables
x = [m.add_var(var_type=mip.BINARY) for i in range(n1)]

# constraints
for i in range(16, 31):
    for j in range(i + 1, 31):
        dist = distance(i, j)
        m.add_constr(mip.xsum(x[k] for k in range(i - 15, j - 16)) >= (1 / 1000) * (dist - 50))

# objective function
m.objective = mip.minimize(mip.xsum(inst_cost[i] * x[i] for i in range(n1)))

# model optimization
m.optimize()

# print the result
n_installed = 0
for i in range(len(x)):
    n_installed += x[i].x

print("Number of charging stations installed:", n_installed)
print("Total cost:", m.objective_value)
for i in range(n1):
    if x[i].x >= 0.99:
        print(f"Charging station installed at POI {i+1}'")


In [ ]:
# Solution visualization on the graph: POIs with charging stations in green

highlighted_nodes = [17, 19, 20, 21, 23, 25, 27, 28]

node_colors2 = ['green' if node in highlighted_nodes else 'yellow' for node in g2.nodes()]

labels = {}
for i in range(1, n + 1):
    labels[i] = str(i)
for i in range(n + 1, n + n1 + 1):
    labels[i] = f"{i - n}'"
labels[s] = 's'
labels[t] = 't'

plt.figure(figsize=(25, 6))
nx.draw(g2, pos=coord2, with_labels=True, labels=labels, node_color=node_colors2, node_size=500, font_size=10)
edge_labels = nx.get_edge_attributes(g2, 'weight')
nx.draw_networkx_edge_labels(g2, pos=coord2, edge_labels=edge_labels, font_color='blue')
plt.show()


In [ ]:
# Comparison of obtained results

plt.figure(figsize=(40, 8))
plt.suptitle("Comparison between the graphs of problem 1 and 2", fontsize=15)

plt.subplot(121)
plt.title("Graph of problem 1")
nx.draw(g1, pos=coord1, with_labels=True, labels=labels, node_color=node_colors1, node_size=500, font_size=10)
edge_labels1 = nx.get_edge_attributes(g1, 'weight')
nx.draw_networkx_edge_labels(g1, pos=coord1, edge_labels=edge_labels1, font_color='blue')

plt.subplot(122)
plt.title("Graph of problem 2")
nx.draw(g2, pos=coord2, with_labels=True, labels=labels, node_color=node_colors2, node_size=500, font_size=10)
edge_labels2 = nx.get_edge_attributes(g2, 'weight')
nx.draw_networkx_edge_labels(g2, pos=coord2, edge_labels=edge_labels2, font_color='blue')

plt.show()


## Comment

Do you obtain the same solution? Why?

----- Francesco Pisacane -----

### The solutions obtained are different.

In the first problem, the result is as follows:  
- Number of charging stations installed: 8.0  
- Total cost: 12348.0  
- Charging stations installed at POIs: 2', 4', 5', 7', 8', 10', 12', 14'  

In the second problem, the result is as follows:  
- Number of charging stations installed: 8.0  
- Total cost: 12924.0  
- Charging stations installed at POIs: 2', 4', 5', 6', 8', 10', 12', 13'  

On closer inspection, conceptually, the two problems are essentially equivalent, and these differences are due to the varying data values involved. In fact, due to the territory’s morphology, energy consumption varies, and consequently, so does the optimal choice of POIs where charging stations should be installed.
\\

### Explanation of the Code/Logic Used:

- **Graph Visualization**: I based my approach on the `networkx` module, which we explored during lab sessions. After defining the set of nodes and edges, assigning them coordinates and corresponding weights, I simply defined the labels and used the module’s functions to display all necessary information on the graph (highlighting in green the POIs where charging stations should be installed).

- **Model**: I based my approach on the `mip` module, also reviewed in lab sessions. I first approached the problem on paper as a standard linear programming problem and then translated it into Python:

1) **Variables** -> I needed to decide which POIs should have charging stations installed, so I introduced a binary variable `X` for each POI (1 if a station is installed at that POI, 0 otherwise).

\begin{equation}
x_i =
\begin{cases}
  1\ if\ there\ is\ a\ station\ at\ i \\
  0\ otherwise
\end{cases}
\end{equation} \\

Implementation with `mip`, using the `add_var` function: \\
```
x = [m.add_var(var_type = mip.BINARY) for i in range(n1)]
```




2) Constraints -> The only constraint involved is the maximum consumption between two consecutive charging stations. To express this mathematically, we used the following linear expression: \\
Let $d_{i, j}$ be the total consumption between POIs i and j, then \\
∀i, j ∈ H it must hold that: $\sum_{i < k < j} x_k \geq \frac{1}{1000}*(d_{i, j} - Δ)$

Note: The right term becomes a positive value only when the consumption between i and j exceeds the limit Δ; otherwise, it is always negative. Additionally, by scaling with a large enough constant M (in this case, 1000 seemed reasonable), we force this value to be between 0 and 1 in absolute terms. \\
In this way, if the consumption between any two POIs (i, j) exceeds the limit of 50, we enforce the presence of at least one charging station at POIs located between i and j, excluding the endpoints. (Note: we simply translated the concept of implication into an inequality).

Implementation with `mip`, using the `add_constr` function: \\
```
m.add_constr(mip.xsum(x[k] for k in range(i - 15, j - 16)) >= (1 / 1000) * (dist - 50))
```

3)  Objective Function -> The goal was to minimize installation costs. This translates into a linear objective function in the system variables given by:\\
$min{\sum_{i \in H} x_i*c_i}$ \\
where $c_i$ represents the installation costs at the i-th POI. \\

Implementation with `mip`, using the `minimize` function: \\
```
m.objective = mip.minimize(mip.xsum(inst_cost[i] * x[i] for i in range(n1)))
```